In [2]:
from google.colab import drive

drive.mount('/content/gdrive/')

TIMEOUT: ignored

In [ ]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

from torchvision import transforms
from torchvision.models import resnet50

In [ ]:
data_dir = '/content/gdrive/MyDrive/data/dirty mnist/'

os.chdir(data_dir)
print('current dir:', os.getcwd())

current dir: /content/gdrive/MyDrive/data/dirty mnist


In [ ]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike, # os.PathLike: 파일 시스템 경로를 나타내는 객체
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            print("reader = ", reader)
            for row in reader:
                print("row = ", row)
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        print("image_ids = ", image_ids)
        print("type(image_ids) = ", type(image_ids))
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

In [ ]:
transforms_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
trainset = MnistDataset('')